In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as se
import mysql.connector
from datetime import datetime

class Personal_Finance:
    def mainfx(self):
        data = pd.read_csv('/home/happy/Downloads/spending_patterns_detailed.csv')
        # print(data)
        # print(data.sample(10))

        # 1. Ensure the Transaction Date is datetime
        data['Transaction Date'] = pd.to_datetime(data['Transaction Date'], errors='coerce')

        # 2. Extract the month as a Period
        data['Month'] = data['Transaction Date'].dt.to_period('M')

        # 3. Group by Month and calculate total income
        monthly_income = data.groupby('Month')['Total Spent'].sum().reset_index()

        # 4. Convert Period to Timestamp for readability (optional)
        monthly_income['Month'] = monthly_income['Month'].dt.to_timestamp()


        # 5. Display the result
        print(monthly_income)

        # BAR PLOT USING SEABORN
        se.set_theme(style="whitegrid")

        plt.figure(figsize=(10, 6))
        se.barplot(data=monthly_income, x='Month', y='Total Spent', color='skyblue')

        plt.title('Monthly Income')
        plt.xlabel('Month')
        plt.ylabel('Total Income')
        plt.xticks(rotation=45)  # Rotate x-axis labels for readability

        plt.tight_layout()
        plt.show()


        # Show total spent per category
      
       
        selected_month = '2024-03'   # e.g., March 2024
        # selected_month = None       # Uncomment to use full dataset

        # ---------- FILTER DATA ----------
        if selected_month:
            filtered_data = data[data['Month'] == selected_month]
            title = f'Spending by Category – {selected_month}'
        else:
            filtered_data = data
            title = 'Total Spending by Category'

        # ---------- AGGREGATE ----------
        category_spending = (
            filtered_data.groupby('Category')['Total Spent']
            .sum()
            .sort_values(ascending=False)
        )

        # # ---------- PLOTTING ----------
        plt.figure(figsize=(9, 9))
        colors = plt.cm.Set3.colors  # Clean color palette

        def make_autopct(values):
            def my_autopct(pct):
                total = sum(values)
                val = int(round(pct*total/100.0))
                return f'{pct:.1f}%\n({val})'
            return my_autopct

        plt.pie(
            category_spending.values,
            labels=category_spending.index,
            autopct=make_autopct(category_spending.values),
            startangle=90,
            colors=colors,
            textprops={'fontsize': 10}
        )

        plt.title(title, fontsize=14)
        plt.axis('equal')  # Equal aspect ratio ensures a perfect circle
        plt.tight_layout()
        plt.show()

        
        # Monthly mean and std daviation
        print(monthly_income['Total Spent'].mean())
        print(monthly_income['Total Spent'].std())

        
        # PUT DATA INTO SQL DATABASES

        # === Step 1: Load CSV ===
        csv_path = '/home/happy/Downloads/spending_patterns_detailed.csv'  # change this to your CSV path
        df = pd.read_csv(csv_path)

        # === Step 2: Connect to MySQL ===
        conn = mysql.connector.connect(
            host='localhost',
            user='root',
            password='Dkq*******'
        )
        cursor = conn.cursor()

        # === Step 3: Create Database ===
        cursor.execute("CREATE DATABASE IF NOT EXISTS sales_db")
        cursor.execute("USE sales_db")

        # === Step 4: Create Table ===
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS transactions (
            Customer_ID VARCHAR(50),
            Category VARCHAR(100),
            Item VARCHAR(100),
            Quantity INT,
            Price_Per_Unit FLOAT,
            Total_Spent FLOAT,
            Payment_Method VARCHAR(50),
            Location VARCHAR(100),
            Transaction_Date DATE,
            Month VARCHAR(20)
        )
        """)

        # Ensure the DataFrame columns match the table columns and order

        columns = [
            'Customer_ID', 'Category', 'Item', 'Quantity', 'Price_Per_Unit',
            'Total_Spent', 'Payment_Method', 'Location', 'Transaction_Date', 'Month'
        ]
        # If 'Transaction Date' is the column in CSV, rename it for SQL insert

        if 'Transaction Date' in df.columns:
            df = df.rename(columns={'Transaction Date': 'Transaction_Date'})

        # If 'Total Spent' is the column in CSV, rename it for SQL insert
        if 'Total Spent' in df.columns:
            df = df.rename(columns={'Total Spent': 'Total_Spent'})
            
        # If 'Price Per Unit' is the column in CSV, rename it for SQL insert
        if 'Price Per Unit' in df.columns:
            df = df.rename(columns={'Price Per Unit': 'Price_Per_Unit'})

        # Only select the columns needed and fill missing with None
        df = df.reindex(columns=columns)

        # Convert dates to string in 'YYYY-MM-DD' format for SQL
        df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], errors='coerce').dt.strftime('%Y-%m-%d')

        # === Step 5: Insert Data ===
        for row in df.itertuples(index=False, name=None):
            cursor.execute("""
            INSERT INTO transactions (
                Customer_ID, Category, Item, Quantity, Price_Per_Unit,
                Total_Spent, Payment_Method, Location, Transaction_Date, Month
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, row)

        conn.commit()
        cursor.close()
        conn.close()
                
if __name__=='__main__':
    cls = Personal_Finance()
    cls.mainfx()